<a href="https://colab.research.google.com/github/Asigen93/finetuningGAN/blob/main/tes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install versi PyTorch yang stabil untuk CUDA Colab**

In [1]:
!pip install torch==2.8.0 torchvision==0.23.0 torchaudio==2.8.0 --index-url https://download.pytorch.org/whl/cu126

Looking in indexes: https://download.pytorch.org/whl/cu126


# **Install library pendukung untuk GAN dan visualisasi**

In [2]:
!pip install numpy matplotlib tqdm pillow seaborn
!pip install einops --quiet

# **Import dan cek versi PyTorch**

In [3]:
import torch
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

PyTorch version: 2.8.0+cu126
CUDA available: True
GPU: Tesla T4


# **Siapkan Dataset**

In [4]:
from google.colab import drive
drive.mount('/content/drive')

import os
data_dir = '/content/drive/MyDrive/ayam'
print("Total data:", len(os.listdir(data_dir)))

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import os, shutil

base_dir = '/content/drive/MyDrive/ayam'
new_dir = os.path.join(base_dir, 'class0')

if not os.path.exists(new_dir):
    os.makedirs(new_dir)
    for file in os.listdir(base_dir):
        if file.lower().endswith(('.jpg', '.png', '.jpeg')):
            shutil.move(os.path.join(base_dir, file), os.path.join(new_dir, file))

print("✅ Semua gambar dipindahkan ke:", new_dir)

In [ ]:
dataset = datasets.ImageFolder(root=base_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# **Definisikan Arsitektur GAN (Generator + Discriminator)**

In [ ]:
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, z_dim=100, img_channels=3, features_g=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.ConvTranspose2d(z_dim, features_g*8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(features_g*8),
            nn.ReLU(True),

            nn.ConvTranspose2d(features_g*8, features_g*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_g*4),
            nn.ReLU(True),

            nn.ConvTranspose2d(features_g*4, features_g*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_g*2),
            nn.ReLU(True),

            nn.ConvTranspose2d(features_g*2, features_g, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_g),
            nn.ReLU(True),

            nn.ConvTranspose2d(features_g, img_channels, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        return self.net(x)

class Discriminator(nn.Module):
    def __init__(self, img_channels=3, features_d=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(img_channels, features_d, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(features_d, features_d*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_d*2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(features_d*2, features_d*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_d*4),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(features_d*4, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

# **Latih Model GAN**

In [ ]:
import torch.optim as optim
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
z_dim = 100
lr = 0.0002
epochs = 50

gen = Generator(z_dim).to(device)
disc = Discriminator().to(device)
opt_gen = optim.Adam(gen.parameters(), lr=lr, betas=(0.5, 0.999))
opt_disc = optim.Adam(disc.parameters(), lr=lr, betas=(0.5, 0.999))
criterion = nn.BCELoss()

fixed_noise = torch.randn(32, z_dim, 1, 1).to(device)

for epoch in range(epochs):
    for real, _ in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
        real = real.to(device)
        batch_size = real.size(0)
        noise = torch.randn(batch_size, z_dim, 1, 1).to(device)
        fake = gen(noise)

        ### Train Discriminator
        disc_real = disc(real).reshape(-1)
        lossD_real = criterion(disc_real, torch.ones_like(disc_real))
        disc_fake = disc(fake.detach()).reshape(-1)
        lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
        lossD = (lossD_real + lossD_fake) / 2
        opt_disc.zero_grad()
        lossD.backward()
        opt_disc.step()

        ### Train Generator
        output = disc(fake).reshape(-1)
        lossG = criterion(output, torch.ones_like(output))
        opt_gen.zero_grad()
        lossG.backward()
        opt_gen.step()

    print(f"Epoch [{epoch+1}/{epochs}] | Loss D: {lossD:.4f} | Loss G: {lossG:.4f}")

# **Visualisasi Hasil Gambar**

In [ ]:
import matplotlib.pyplot as plt
import torchvision.utils as vutils

with torch.no_grad():
    fake_images = gen(fixed_noise).detach().cpu()

plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Generated Images")
plt.imshow(
    vutils.make_grid(fake_images, padding=2, normalize=True).permute(1, 2, 0)
)
plt.show()

# **Simpan Model (opsional)**

In [ ]:
torch.save(gen.state_dict(), "generator.pth")
torch.save(disc.state_dict(), "discriminator.pth")